In [1]:
# @title Installing
!pip install hazm

In [28]:
# @title Import
import numpy as np
from sklearn import datasets
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from hazm import Normalizer, Stemmer, word_tokenize, stopwords_list
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import text
from gensim.models import FastText, Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
import fasttext
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split

In [4]:
# @title Loading Data
data = pd.read_csv('/content/Snappfood - Sentiment Analysis.csv' , on_bad_lines='skip' , delimiter='\t')
data.head()

,Unnamed: 0,comment,label,label_id
0,NaN,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,SAD,1.0
1,NaN,قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...,HAPPY,0.0
2,NaN,قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...,SAD,1.0
3,NaN,عالللی بود همه چه درست و به اندازه و کیفیت خوب...,HAPPY,0.0
4,NaN,شیرینی وانیلی فقط یک مدل بود.,HAPPY,0.0


In [5]:
# @title Preprocceing
data = data[['comment' , 'label' , 'label_id']]
data.dropna(inplace=True)
data.drop_duplicates(keep='first', inplace=True)
data['label_id'] = data['label_id'].astype(int)
data.head()

<ipython-input-5-23a78615a10e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.dropna(inplace=True)


,comment,label,label_id
0,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,SAD,1
1,قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...,HAPPY,0
2,قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...,SAD,1
3,عالللی بود همه چه درست و به اندازه و کیفیت خوب...,HAPPY,0
4,شیرینی وانیلی فقط یک مدل بود.,HAPPY,0


In [6]:
# @title 1. Normalize
normalizer = Normalizer()
data['comment_normalized'] = data['comment'].apply(normalizer.normalize)

print(data['comment_normalized'])

0          واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح
1        قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...
2        قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...
3        عالی بود همه چه درست و به اندازه و کیفیت خوب، ...
4                            شیرینی وانیلی فقط یک مدل بود.
                               ...                        
69995    سلام من به فاکتور غذاهایی که سفارش میدم احتیاج...
69996    سایز پیتزا نسبت به سفارشاتی که قبلا گذشتم کم ش...
69997    من قارچ اضافه رو اضافه کرده بودم بودم اما اگر ...
69998    همرو بعد ۲ ساعت تاخیر اشتباه آوردن پولشم رفت ر...
69999                                  فلفلش خیلی تند بود.
Name: comment_normalized, Length: 69480, dtype: object


In [7]:
# @title 2. Tokenize
data['comment_tokenized'] = data['comment_normalized'].apply(word_tokenize)

print(data['comment_tokenized'])

0        [واقعا, حیف, وقت, که, بنویسم, سرویس, دهیتون, ش...
1        [قرار, بود, ۱, ساعته, برسه, ولی, نیم, ساعت, زو...
2        [قیمت, این, مدل, اصلا, با, کیفیتش, سازگاری, ند...
3        [عالی, بود, همه, چه, درست, و, به, اندازه, و, ک...
4                   [شیرینی, وانیلی, فقط, یک, مدل, بود, .]
                               ...                        
69995    [سلام, من, به, فاکتور, غذاهایی, که, سفارش, مید...
69996    [سایز, پیتزا, نسبت, به, سفارشاتی, که, قبلا, گذ...
69997    [من, قارچ, اضافه, رو, اضافه, کرده_بودم, بودم, ...
69998    [همرو, بعد, ۲, ساعت, تاخیر, اشتباه, آوردن, پول...
69999                           [فلفلش, خیلی, تند, بود, .]
Name: comment_tokenized, Length: 69480, dtype: object


In [8]:
# @title 3. Stemming
stemmer = Stemmer()
data['comment_stemmed'] = data['comment_tokenized'].apply(lambda tokens: [stemmer.stem(token) for token in tokens])

print(data['comment_stemmed'])

0        [واقعا, حیف, وق, که, بنویس, سرویس, دهیتون, شده...
1        [قرار, بود, ۱, ساعته, برسه, ول, ن, ساع, زود, ا...
2        [قیم, این, مدل, اصلا, با, کیفیت, سازگار, نداره...
3        [عال, بود, همه, چه, درس, و, به, اندازه, و, کیف...
4                     [شیرین, وانیل, فقط, یک, مدل, بود, .]
                               ...                        
69995    [سلا, من, به, فاکتور, غذا, که, سفار, مید, احتی...
69996    [سایز, پیتزا, نسب, به, سفارشات, که, قبلا, گذ, ...
69997    [من, قارچ, اضافه, رو, اضافه, کرده_بود, بود, ام...
69998    [همرو, بعد, ۲, ساع, تاخیر, اشتباه, آوردن, پول,...
69999                             [فلفل, خیل, تند, بود, .]
Name: comment_stemmed, Length: 69480, dtype: object


In [9]:
# @title 4. Deleting Stop Words
stopwords = set(stopwords_list())
data['comment_filtered'] = data['comment_stemmed'].apply(lambda tokens: [token for token in tokens if token not in stopwords])

print(data['comment_filtered'])

0           [واقعا, حیف, وق, بنویس, سرویس, دهیتون, افتضاح]
1        [قرار, ۱, ساعته, برسه, ول, ن, ساع, زود, موقع, ...
2        [قیم, مدل, اصلا, کیفیت, سازگار, نداره, ،, ظاهر...
3        [عال, درس, اندازه, کیف, ،, امیداور, کیفیتتون, ...
4                                   [شیرین, وانیل, مدل, .]
                               ...                        
69995    [سلا, فاکتور, غذا, سفار, مید, احتیاج, موضوع, ق...
69996             [سایز, پیتزا, نسب, سفارشات, قبلا, گذ, ک]
69997    [قارچ, اضافه, اضافه, ه, اضافه, نمی‌کرد, نمیدون...
69998    [همرو, ۲, ساع, تاخیر, اشتباه, آوردن, پول, رف, ...
69999                                  [فلفل, خیل, تند, .]
Name: comment_filtered, Length: 69480, dtype: object


In [13]:
# @title TF-IDF Vectorize
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(data['comment'])
tfidf_embeddings = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())
print("TF-IDF Embeddings:")
print(tfidf_embeddings)

TF-IDF Embeddings:
        aa  aaaallliii  aali  aalllii   ab  ablimo  about  acting  adam  \
0      0.0         0.0   0.0      0.0  0.0     0.0    0.0     0.0   0.0   
1      0.0         0.0   0.0      0.0  0.0     0.0    0.0     0.0   0.0   
2      0.0         0.0   0.0      0.0  0.0     0.0    0.0     0.0   0.0   
3      0.0         0.0   0.0      0.0  0.0     0.0    0.0     0.0   0.0   
4      0.0         0.0   0.0      0.0  0.0     0.0    0.0     0.0   0.0   
...    ...         ...   ...      ...  ...     ...    ...     ...   ...   
69475  0.0         0.0   0.0      0.0  0.0     0.0    0.0     0.0   0.0   
69476  0.0         0.0   0.0      0.0  0.0     0.0    0.0     0.0   0.0   
69477  0.0         0.0   0.0      0.0  0.0     0.0    0.0     0.0   0.0   
69478  0.0         0.0   0.0      0.0  0.0     0.0    0.0     0.0   0.0   
69479  0.0         0.0   0.0      0.0  0.0     0.0    0.0     0.0   0.0   

       adasi  ...  ۹۵۹۵   ۹۶   ۹۷  ۹۷۰۰۰   ۹۸  ۹۸درصد   ۹۹  ۹۹۰  ۹۹۰۰  ۹۹۰۱۰۹  


In [14]:
# @title Count Vectorize
data['comment'] = data['comment'].apply(word_tokenize)
data['comment'] = data['comment'].apply(' '.join)
count_vectorizer = CountVectorizer()
count_matrix = count_vectorizer.fit_transform(data['comment'])
count_embeddings = pd.DataFrame(count_matrix.toarray(), columns=count_vectorizer.get_feature_names_out())
print("Count Vectorized Embeddings:")
print(count_embeddings)

Count Vectorized Embeddings:
       aa  aaaallliii  aali  aalllii  ab  ablimo  about  acting  adam  adasi  \
0       0           0     0        0   0       0      0       0     0      0   
1       0           0     0        0   0       0      0       0     0      0   
2       0           0     0        0   0       0      0       0     0      0   
3       0           0     0        0   0       0      0       0     0      0   
4       0           0     0        0   0       0      0       0     0      0   
...    ..         ...   ...      ...  ..     ...    ...     ...   ...    ...   
69475   0           0     0        0   0       0      0       0     0      0   
69476   0           0     0        0   0       0      0       0     0      0   
69477   0           0     0        0   0       0      0       0     0      0   
69478   0           0     0        0   0       0      0       0     0      0   
69479   0           0     0        0   0       0      0       0     0      0   

       ...

In [15]:
# @title FastText Embeddings
data['comment'].to_csv('preprocessed_text.txt', index=False, header=False)
model = fasttext.train_unsupervised('preprocessed_text.txt', model='skipgram')
fasttext_embeddings = []
for comment in data['comment']:
    embedding = model.get_sentence_vector(comment)
    fasttext_embeddings.append(embedding)

fasttext_embeddings = pd.DataFrame(fasttext_embeddings)
data_with_embeddings = pd.concat([data, fasttext_embeddings], axis=1)
print("Data with FastText Embeddings:")
print(data_with_embeddings)

Data with FastText Embeddings:
                                                 comment  label  label_id  \
0        واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح    SAD       1.0   
1      قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...  HAPPY       0.0   
2      قیمت این مدل اصلا با کیفیتش سازگاری نداره ، فق...    SAD       1.0   
3      عالللی بود همه چه درست و به اندازه و کیفیت خوب...  HAPPY       0.0   
4                         شیرینی وانیلی فقط یک مدل بود .  HAPPY       0.0   
...                                                  ...    ...       ...   
69168                                                NaN    NaN       NaN   
69228                                                NaN    NaN       NaN   
69400                                                NaN    NaN       NaN   
69403                                                NaN    NaN       NaN   
69467                                                NaN    NaN       NaN   

                                      commen

In [16]:
# @title Word2Vec
data['comment'] = data['comment'].apply(str)
data['comment'] = data['comment'].apply(word_tokenize)

model = Word2Vec(sentences=data['comment'], vector_size=100, window=5, min_count=1, workers=4)
word2vec_embeddings = []
for comment in data['comment']:
    embedding = []
    for word in comment:
        if word in model.wv:
            embedding.append(model.wv[word])
    word2vec_embeddings.append(np.mean(embedding, axis=0))
word2vec_embeddings = pd.DataFrame(word2vec_embeddings)

data_with_embeddings = pd.concat([data, word2vec_embeddings], axis=1)

print("Data with Word2Vec Embeddings:")
print(data_with_embeddings)

Data with Word2Vec Embeddings:
                                                 comment  label  label_id  \
0      [واقعا, حیف, وقت, که, بنویسم, سرویس, دهیتون, ش...    SAD       1.0   
1      [قرار, بود, ۱, ساعته, برسه, ولی, نیم, ساعت, زو...  HAPPY       0.0   
2      [قیمت, این, مدل, اصلا, با, کیفیتش, سازگاری, ند...    SAD       1.0   
3      [عالللی, بود, همه, چه, درست, و, به, اندازه, و,...  HAPPY       0.0   
4                 [شیرینی, وانیلی, فقط, یک, مدل, بود, .]  HAPPY       0.0   
...                                                  ...    ...       ...   
69168                                                NaN    NaN       NaN   
69228                                                NaN    NaN       NaN   
69400                                                NaN    NaN       NaN   
69403                                                NaN    NaN       NaN   
69467                                                NaN    NaN       NaN   

                                      commen

In [24]:
# @title Forest Classifier

# تقسیم داده‌های ورودی به داده‌های آموزش و آزمون
X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix, data['label'], test_size=0.2, random_state=42)

# آموزش مدل روش تصادفی Forest Classifier
model = RandomForestClassifier()
model.fit(X_train, y_train)

# ارزیابی مدل
accuracy = model.score(X_test, y_test)
print("Accuracy:", accuracy)

Accuracy: 0.8486614853195165


In [25]:
# @title Logistic Regression

# تقسیم داده‌های ورودی به داده‌های آموزش و آزمون
X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix, data['label'], test_size=0.2, random_state=42)

# آموزش مدل رگرسیون لجستیک
model = LogisticRegression()
model.fit(X_train, y_train)

# ارزیابی مدل
accuracy = model.score(X_test, y_test)
print("Accuracy:", accuracy)

Accuracy: 0.8566493955094991


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [30]:
# @title Naive Bayes

# تقسیم داده‌های ورودی به داده‌های آموزش و آزمون
X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix, data['label'], test_size=0.2, random_state=42)

# آموزش مدل ناییو بیز
model = MultinomialNB()
model.fit(X_train, y_train)

# ارزیابی مدل
accuracy = model.score(X_test, y_test)
print("Accuracy:", accuracy)

Accuracy: 0.8354202648244099
